In [137]:
# Import Libraries and Dependencies
import pandas as pd

### 1. Combine and Clean the Data
#### Import CSVs

In [138]:
# Read the CSV files into DataFrames.
df_2020 = pd.read_csv('athletic_sales_2020.csv')  
df_2021 = pd.read_csv('athletic_sales_2021.csv')


In [139]:
# Display the 2020 sales DataFrame
print("2020 Sales DataFrame:\n", df_2020.head())

2020 Sales DataFrame:
       retailer  retailer_id invoice_date     region         state  \
0  Foot Locker      1185732       1/1/20  Northeast      New York   
1  Foot Locker      1185732       1/1/20  Northeast  Pennsylvania   
2  Foot Locker      1185732       1/1/20  Northeast  Pennsylvania   
3  Foot Locker      1185732       1/1/20  Northeast      New York   
4  Foot Locker      1185732       1/1/20  Northeast  Pennsylvania   

           city                product  price_per_unit  units_sold  \
0      New York  Men's Street Footwear              50        1200   
1  Philadelphia        Women's Apparel              68          83   
2  Philadelphia        Women's Apparel              75         275   
3      New York  Men's Street Footwear              34         384   
4  Philadelphia        Women's Apparel              53          83   

   total_sales  operating_profit sales_method  
0       600000         300000.00     In-store  
1         5644           2426.92       Online

In [140]:
# Display the 2021 sales DataFrame
print("\n2021 Sales DataFrame:\n", df_2021.head())


2021 Sales DataFrame:
     retailer  retailer_id invoice_date   region       state           city  \
0  West Gear      1128299       1/1/21     West  California  San Francisco   
1  West Gear      1128299       1/1/21     West  California  San Francisco   
2     Kohl's      1189833       1/1/21  Midwest     Montana       Billings   
3     Kohl's      1189833       1/1/21  Midwest     Montana       Billings   
4  West Gear      1128299       1/1/21     West  California  San Francisco   

                   product  price_per_unit  units_sold  total_sales  \
0  Men's Athletic Footwear              65         750       487500   
1  Men's Athletic Footwear              51         233        11883   
2            Men's Apparel              50         275       137500   
3            Men's Apparel              47          77         3619   
4  Men's Athletic Footwear              64         225        14400   

   operating_profit sales_method  
0         121875.00       Outlet  
1         

#### Check the data types of each DataFrame

In [141]:
# Check the 2020 sales data types.
print("\n2020 Sales Data Types:\n", df_2020.dtypes)



2020 Sales Data Types:
 retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


In [142]:
# Check the 2021 sales data types.
print("\n2021 Sales Data Types:\n", df_2021.dtypes)



2021 Sales Data Types:
 retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


#### Combine the sales data by rows.

In [143]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index.
combined_sales = pd.concat([df_2020, df_2021], axis=0).reset_index(drop=True)


In [144]:
# Check if any values are null.
print("\nNull Values Check:\n", combined_sales.isnull().sum())



Null Values Check:
 retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64


In [145]:
# Check the data type of each column
print("\nCombined DataFrame Data Types:\n", combined_sales.dtypes)



Combined DataFrame Data Types:
 retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


In [146]:
# Convert the "invoice_date" to a datetime datatype
combined_sales['invoice_date'] = pd.to_datetime(combined_sales['invoice_date'], format='%m/%d/%Y', errors='coerce')

In [147]:
# Confirm that the "invoice_date" data type has been changed.
print("\nUpdated Data Types After Conversion:\n", combined_sales.dtypes)



Updated Data Types After Conversion:
 retailer                    object
retailer_id                  int64
invoice_date        datetime64[ns]
region                      object
state                       object
city                        object
product                     object
price_per_unit               int64
units_sold                   int64
total_sales                  int64
operating_profit           float64
sales_method                object
dtype: object


### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [148]:
# Show the number products sold for region, state, and city.
# Rename the sum to "Total_Products_Sold".
# Show the top 5 results.
products_sold = (
    combined_sales.groupby(['region', 'state', 'city'])['units_sold']
    .sum()
    .reset_index(name='Total_Products_Sold')
)
print("\nTop 5 Regions by Products Sold:\n", products_sold.sort_values(by='Total_Products_Sold', ascending=False).head())




Top 5 Regions by Products Sold:
        region       state           city  Total_Products_Sold
21  Northeast    New York       New York               111954
33      South       Texas        Houston                90322
44       West  California  San Francisco                85478
43       West  California    Los Angeles                76384
34  Southeast     Florida          Miami                73135


#### Using `pivot_table`

In [149]:
# Show the number products sold for region, state, and city.
# Rename the "units_sold" column to "Total_Products_Sold"
# Show the top 5 results.
products_sold = (
    combined_sales.pivot_table(
        index=['region', 'state', 'city'],  
        values='units_sold',  
        aggfunc='sum'  
    )
    .reset_index()  
    .rename(columns={'units_sold': 'Total_Products_Sold'}) 
)
print("\nTop 5 Regions by Products Sold Using Pivot Table:\n", products_sold.sort_values(by='Total_Products_Sold', ascending=False).head())


Top 5 Regions by Products Sold Using Pivot Table:
        region       state           city  Total_Products_Sold
21  Northeast    New York       New York               111954
33      South       Texas        Houston                90322
44       West  California  San Francisco                85478
43       West  California    Los Angeles                76384
34  Southeast     Florida          Miami                73135


### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [150]:
# Show the total sales for the products sold for each region, state, and city.
# Rename the "total_sales" column to "Total Sales"
# Show the top 5 results.
total_sales = (
    combined_sales.groupby(['region', 'state', 'city'])['total_sales']
    .sum()
    .reset_index(name='Total Sales')
)
print("\nTop 5 Regions by Total Sales:\n", total_sales.sort_values(by='Total Sales', ascending=False).head())



Top 5 Regions by Total Sales:
        region           state           city  Total Sales
21  Northeast        New York       New York     39801235
44       West      California  San Francisco     33973228
34  Southeast         Florida          Miami     31600863
39  Southeast  South Carolina     Charleston     29285637
35  Southeast         Florida        Orlando     27682851


#### Using `pivot_table`

In [151]:
# Show the total sales for the products sold for each region, state, and city.
# Optional: Rename the "total_sales" column to "Total Sales"
# Show the top 5 results.
total_sales = (
    combined_sales.pivot_table(
        index=['region', 'state', 'city'], 
        values='total_sales', 
        aggfunc='sum' 
    )
    .reset_index()  
    .rename(columns={'total_sales': 'Total Sales'}) 
)
print("\nTop 5 Regions by Total Sales Using Pivot Table:\n", total_sales.sort_values(by='Total Sales', ascending=False).head())


Top 5 Regions by Total Sales Using Pivot Table:
        region           state           city  Total Sales
21  Northeast        New York       New York     39801235
44       West      California  San Francisco     33973228
34  Southeast         Florida          Miami     31600863
39  Southeast  South Carolina     Charleston     29285637
35  Southeast         Florida        Orlando     27682851


### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [152]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Rename the "total_sales" column to "Total Sales"
# Show the top 5 results.
retailer_sales = (
    combined_sales.groupby(['retailer', 'region', 'state', 'city'])['total_sales']
    .sum()
    .reset_index(name='Total Sales')
)
print("\nTop 5 Retailers by Sales:\n", retailer_sales.sort_values(by='Total Sales', ascending=False).head())



Top 5 Retailers by Sales:
         retailer     region           state           city  Total Sales
103    West Gear       West      California  San Francisco     32794405
50        Kohl's       West      California    Los Angeles     25127160
22   Foot Locker  Northeast        New York       New York     25008568
109    West Gear       West      Washington        Seattle     24862675
33   Foot Locker  Southeast  South Carolina     Charleston     24822280


#### Using `pivot_table`

In [153]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Optional: Rename the "total_sales" column to "Total Sales"
# Show the top 5 results.
retailer_sales = (
    combined_sales.pivot_table(
        index=['retailer', 'region', 'state', 'city'], 
        values='total_sales',  
        aggfunc='sum'  
    )
    .reset_index()  
    .rename(columns={'total_sales': 'Total Sales'})  
)
print("\nTop 5 Retailers by Sales Using Pivot Table:\n", retailer_sales.sort_values(by='Total Sales', ascending=False).head())


Top 5 Retailers by Sales Using Pivot Table:
         retailer     region           state           city  Total Sales
103    West Gear       West      California  San Francisco     32794405
50        Kohl's       West      California    Los Angeles     25127160
22   Foot Locker  Northeast        New York       New York     25008568
109    West Gear       West      Washington        Seattle     24862675
33   Foot Locker  Southeast  South Carolina     Charleston     24822280


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [154]:
# Filter the sales data to get the women's athletic footwear sales data.
womens_footwear = combined_sales[combined_sales['product'] == "Women's Athletic Footwear"]

#### Using `groupby`

In [155]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
# Show the top 5 results.
womens_units_sold = (
    womens_footwear.groupby(['retailer', 'region', 'state', 'city'])['units_sold']
    .sum()
    .reset_index(name='Womens_Footwear_Units_Sold')
)
print("\nTop 5 Women's Footwear Units Sold:\n", womens_units_sold.sort_values(by='Womens_Footwear_Units_Sold', ascending=False).head())



Top 5 Women's Footwear Units Sold:
           retailer     region           state           city  \
101      West Gear       West      California  San Francisco   
22     Foot Locker  Northeast        New York       New York   
49          Kohl's       West      California    Los Angeles   
33     Foot Locker  Southeast  South Carolina     Charleston   
68   Sports Direct      South           Texas         Dallas   

     Womens_Footwear_Units_Sold  
101                       12107  
22                        10996  
49                        10826  
33                         8814  
68                         8790  


#### Using `pivot_table`

In [156]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
# Show the top 5 results.
womens_units_sold = (
    womens_footwear.pivot_table(
        index=['retailer', 'region', 'state', 'city'],  
        values='units_sold',  
        aggfunc='sum'  
    )
    .reset_index()  
    .rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'})  
)

print("\nTop 5 Women's Footwear Units Sold Using Pivot Table:\n", womens_units_sold.sort_values(by='Womens_Footwear_Units_Sold', ascending=False).head())


Top 5 Women's Footwear Units Sold Using Pivot Table:
           retailer     region           state           city  \
101      West Gear       West      California  San Francisco   
22     Foot Locker  Northeast        New York       New York   
49          Kohl's       West      California    Los Angeles   
33     Foot Locker  Southeast  South Carolina     Charleston   
68   Sports Direct      South           Texas         Dallas   

     Womens_Footwear_Units_Sold  
101                       12107  
22                        10996  
49                        10826  
33                         8814  
68                         8790  


### 6. Determine the Day with the Most Women's Athletic Footwear Sales

In [185]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
# Optional: Rename the "total_sales" column to "Total Sales"
# Show the table.
sales_pivot = combined_sales.pivot_table(
    index='invoice_date',  
    values='total_sales',  
    aggfunc='sum'  
).rename(columns={'total_sales': 'Total Sales'})  

print("\nPivot Table of Total Sales by Date:\n", sales_pivot.head())


Pivot Table of Total Sales by Date:
 Empty DataFrame
Columns: []
Index: []


In [204]:
# Resample the pivot table into daily bins, and get the total sales for each day.


# Sort the resampled pivot table in descending order on "Total Sales".
# Ensure the index is a datetime object



### 7.  Determine the Week with the Most Women's Athletic Footwear Sales

In [205]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
# Sort the resampled pivot table in descending order on "Total Sales".
